In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import joblib
import shap
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Directories
FEATURES_DIR = Path("features")
BASELINE_MODEL_DIR = Path("models")
DL_MODEL_DIR = Path("models/deep_learning")

# Load features and labels
X = pd.read_pickle(FEATURES_DIR / "X_features.pkl")
y = pd.read_pickle(FEATURES_DIR / "y_labels.pkl")

# --- Baseline Model Interpretation (Random Forest + SHAP) ---
rf_model = joblib.load(BASELINE_MODEL_DIR / "baseline_rf_model.pkl")
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X)

# Plot SHAP summary for top class
plt.figure(figsize=(12,6))
shap.summary_plot(shap_values, X, plot_type="bar", show=False)
plt.title("Random Forest Feature Importance (SHAP)")
plt.tight_layout()
plt.show()

# --- Deep Learning Interpretation (ResNet1D) ---
resnet_model = load_model(DL_MODEL_DIR / "resnet1d_model.h5")
# Using DeepExplainer on a small sample due to compute
sample_idx = np.random.choice(len(X), 500, replace=False)
X_sample = X.iloc[sample_idx].values
X_sample_reshaped = np.expand_dims(X_sample, axis=-1)

dl_explainer = shap.DeepExplainer(resnet_model, X_sample_reshaped)
shap_values_dl = dl_explainer.shap_values(X_sample_reshaped)

# SHAP summary for the first output class
plt.figure(figsize=(12,6))
shap.summary_plot(shap_values_dl[0], X_sample, plot_type="bar", show=False)
plt.title("ResNet1D Feature Importance (SHAP)")
plt.tight_layout()
plt.show()
